In [1]:
import geopandas as gpd
from geopandas.tools import sjoin_nearest

input_file = "data/macon-points.csv"
overture_file = "data/overture-transportation-macon.geojson"
output_file = "data/.output-macon-points-nearest-roads.csv"

In [2]:
points_to_match_gdf = gpd.read_file(input_file, geometry="wkt")
points_to_match_gdf.crs = "epsg:4326"
points_to_match_gdf

,wkt,payload,geometry
0,POINT (-83.6271417 32.8519936),payload1,POINT (-83.62714 32.85199)
1,POINT (-83.6141122 32.8375626),payload2,POINT (-83.61411 32.83756)
2,POINT (-83.6308682 32.8338996),payload3,POINT (-83.63087 32.83390)
3,POINT (-83.6312115 32.8334443),payload4,POINT (-83.63121 32.83344)
4,POINT (-83.6708248 32.8370307),payload5,POINT (-83.67082 32.83703)


In [3]:
overture_gdf = gpd.read_file(overture_file)
overture_gdf.crs = "epsg:4326"
overture_gdf.head()

,id,theme,type,level,road,subType,connectors,geometry
0,66292841,transportation,connector,NaN,NaN,NaN,NaN,POINT (-83.61940 32.85803)
1,66292844,transportation,connector,NaN,NaN,NaN,NaN,POINT (-83.61940 32.85954)
2,9024666,transportation,segment,NaN,{'class': 'residential'},road,"[66292844, 66292841]","LINESTRING (-83.61940 32.85803, -83.61940 32.8..."
3,66292850,transportation,connector,NaN,NaN,NaN,NaN,POINT (-83.71581 32.79509)
4,66292867,transportation,connector,NaN,NaN,NaN,NaN,POINT (-83.71440 32.79499)


In [4]:
# filter the overture features that we want to match against - in this case road segments
segments_gdf = overture_gdf[overture_gdf["type"] == "segment"]
segments_gdf.head()

# project to UTM 17N to get distances in meters
segments_gdf = segments_gdf.to_crs(epsg=32617)
points_to_match_gdf = points_to_match_gdf.to_crs(epsg=32617)

# backup the geometry in a different column name before the join because we want to keep both geometries
segments_gdf["segment_geometry"] = segments_gdf["geometry"]

# spatial join between points and segments - get nearest overture feature, where distance < 100 meters
joined_gdf = sjoin_nearest(points_to_match_gdf, segments_gdf, distance_col="distance", max_distance=100)
joined_gdf
# joined_gdf.explore()

,wkt,payload,geometry,index_right,id,theme,type,level,road,subType,connectors,segment_geometry,distance
0,POINT (-83.6271417 32.8519936),payload1,POINT (254138.713 3637938.158),9716,966254567,transportation,segment,NaN,"{'class': 'residential', 'surface': 'paved'}",road,"[3320243107, 66324040]","LINESTRING (254190.374 3637956.337, 254183.271...",12.693560
1,POINT (-83.6141122 32.8375626),payload2,POINT (255318.799 3636307.400),38293,300000000883,transportation,segment,NaN,"{'class': 'motorway_link', 'surface': 'paved',...",road,"[66300723, 66300705]","LINESTRING (255328.843 3636265.803, 255267.557...",25.107758
2,POINT (-83.6308682 32.8338996),payload3,POINT (253739.858 3635940.118),32098,100000013146,transportation,segment,NaN,{'class': 'residential'},road,"[66348797, 66343102]","LINESTRING (253773.793 3635897.657, 253707.014...",13.213303
3,POINT (-83.6312115 32.8334443),payload4,POINT (253706.456 3635890.423),27700,100000009585,transportation,segment,NaN,"{'class': 'tertiary', 'surface': 'paved', 'res...",road,"[66348797, 66348802]","LINESTRING (253707.014 3635948.527, 253655.091...",34.687320
4,POINT (-83.6708248 32.8370307),payload5,POINT (250007.422 3636381.298),31874,100000012934,transportation,segment,NaN,"{'class': 'tertiary', 'surface': 'paved', 'res...",road,"[66314170, 66314178]","LINESTRING (249997.839 3636366.148, 250033.446...",15.093705


In [5]:
# convert back to wsg84
joined_gdf["geometry"] = joined_gdf["geometry"].to_crs(epsg=4326)
joined_gdf["segment_geometry"] = joined_gdf["segment_geometry"].to_crs(epsg=4326)
# write results to csv file as wkt
joined_gdf["matched_segment_wkt"] = joined_gdf["segment_geometry"].apply(lambda x: x.wkt)
for_output = joined_gdf[["wkt", "payload", "id", "matched_segment_wkt", "distance",]]
for_output.to_csv(output_file)